In [12]:
import cProfile
import io
import os
import requests
import pstats

import numpy as np

import functions


In [ ]:
import laspy
import numpy as np

def extract_vegetation_points(LasData, ndvi_threshold=0.1):
    """
    Extract vegetation points based on classification and NDVI threshold.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.
    
    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """
    
    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]   # High vegetation

    # Calculate NDVI
    red = possible_vegetation_points.red
    nir = possible_vegetation_points.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Filter the points whose NDVI is greater than the threshold
    veg_points = possible_vegetation_points[ndvi > ndvi_threshold]

    return veg_points

def save_vegetation_points_as_las(LasData, output_file, veg_points, ndvi_threshold=0.1):
    """
    Extract vegetation points and save them as a new LAS file.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - output_file (str): Path to the output LAS file.
    - ndvi_threshold (float): NDVI threshold for vegetation points.
    
    Returns:
    - None
    """
    # Extract vegetation points
    
    # Check if any points were filtered
    if len(veg_points) == 0:
        print("No vegetation points found with the given NDVI threshold.")
        return
    
    # Create a new LasData object for the vegetation points
    veg_las_data = laspy.LasData(LasData.header)
    veg_las_data.points = veg_points.points  # Assign the filtered vegetation points

    # Save the vegetation points to a new LAS file
    with laspy.open(output_file, mode="w", header=veg_las_data.header) as writer:
        writer.write_points(veg_las_data.points)
    
    print(f"Vegetation points saved to {output_file}")


file_path = "data/25GN2/25GN2_12.LAZ"
output_las = "data/output_vegetation.las"

In [ ]:
import numpy as np
import laspy

def extract_lasData(LasData, ndvi_threshold=0.1):
    red = LasData.red
    nir = LasData.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Add NDVI as a new dimension/attribute to all points
    if 'NDVI' not in LasData.point_format.dimension_names:
        # Create an array to store NDVI values for all points
        ndvi_dim = np.zeros(LasData.header.point_count, dtype=np.float32)
        ndvi_dim = ndvi  # Assign NDVI values for all points

        # Add this new dimension to LasData
        LasData.add_extra_dim(laspy.ExtraBytesParams(name="NDVI", type=np.float32))
        LasData['NDVI'] = ndvi_dim  # Store NDVI for all points

    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]   # High vegetation


    return LasData

In [ ]:
with laspy.open(file_path, laz_backend=laspy.LazBackend.Lazrs) as las_file:
    las_data = las_file.read()

veg_points = extract_vegetation_points(las_data, ndvi_threshold=0.1)

In [1]:
min_x, max_x = round(las_data.x.min()), round(las_data.x.max())
min_y, max_y = round(las_data.y.min()), round(las_data.y.max())

# Define size of the region
x_length = max_x - min_x
y_length = max_y - min_y

NameError: name 'las_data' is not defined

In [ ]:
def vegetation_raster_idw(LasData, points, resolution, search_radius=1, power=2):
    """
    Create a vegetation raster using Inverse Distance Weighting (IDW) interpolation.

    Parameters:
    - LasData (laspy.LasData): Input LAS data.
    - points (laspy.LasData): Vegetation points to be interpolated.
    - center_point (tuple): Center coordinates (x, y) of the region of interest.
    - resolution (float): Resolution of the raster.
    - half_size (float): Half of the size of the tile (defines the extent of the raster).
    - search_radius (float): Radius for the IDW search. Default is 1.
    - power (float): Power parameter for the IDW algorithm. Default is 2.

    Returns:
    - vege_raster (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Extents of the pc
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())

    # Define size of the region
    x_length = max_x - min_x
    y_length = max_y - min_y

    # Number of rows and columns
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)

    # Initialize raster grid
    vege_raster = np.full((rows, cols), np.nan, dtype=np.float32)

    # Calculate center coords for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)

    if len(points) == 0:
        print("step4, vegetation_raster_idw: There are no vegetation points in the current area.")
        return vege_raster, grid_center_xy

    # Convert LAS points to NumPy array for processing
    points_list = np.array(points.xyz)

    # Interpolation
    functions.interpolation_idw_kdtree(points_list, vege_raster, grid_center_xy, search_radius, power)

    return vege_raster, grid_center_xy

In [ ]:
save_vegetation_points_as_las(las_data, output_las, veg_points, ndvi_threshold=0.1)

In [ ]:
def step4_main(LasData, veg_points, resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False):

    vege_points = extract_vegetation_points(LasData)
    vegetation_data = vegetation_raster_idw(LasData, veg_points, resolution,
                                             search_radius, idw_power)
    veg_raster = vegetation_data[0]
    grid_centers = vegetation_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2
    if write_output:
        transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)
        functions.write_output(vege_points, veg_raster, transform, "../data/chm.tiff")

In [ ]:
step4_main(las_data, veg_points, resolution=0.5, search_radius=1.0, idw_power=2.0, write_output=True, show_fig=False)

In [13]:
import functions
import os
import laspy
import time  # Import time module to track time
from tqdm import tqdm
import numpy as np

def extract_vegetation_points(LasData, ndvi_threshold=0.1):
    """
    Extract vegetation points based on classification and NDVI threshold.

    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.

    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """

    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]  # High vegetation

    # Calculate NDVI
    red = possible_vegetation_points.red
    nir = possible_vegetation_points.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Filter the points whose NDVI is greater than the threshold
    veg_points = possible_vegetation_points[ndvi > ndvi_threshold]

    return veg_points

def chm_creation(LasData, veg_points, output_filename, resolution=0.5, search_radius=1.0, idw_power=2.0):

    veg_points = extract_vegetation_points(LasData)
    vegetation_data = vegetation_raster_idw(LasData, veg_points, resolution,
                                                      search_radius, idw_power)
    veg_raster = vegetation_data[0]
    grid_centers = vegetation_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2

    transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)

    functions.write_output(veg_points, veg_raster, transform, output_filename)


def vegetation_raster_idw(LasData, points, resolution, search_radius=1, power=2):
    """
    Create a vegetation raster using Inverse Distance Weighting (IDW) interpolation.

    Parameters:
    - LasData (laspy.LasData): Input LAS data.
    - points (laspy.LasData): Vegetation points to be interpolated.
    - center_point (tuple): Center coordinates (x, y) of the region of interest.
    - resolution (float): Resolution of the raster.
    - half_size (float): Half of the size of the tile (defines the extent of the raster).
    - search_radius (float): Radius for the IDW search. Default is 1.
    - power (float): Power parameter for the IDW algorithm. Default is 2.

    Returns:
    - vege_raster (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Extents of the pc
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())

    # Define size of the region
    x_length = max_x - min_x
    y_length = max_y - min_y

    # Number of rows and columns
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)

    # Initialize raster grid
    vege_raster = np.full((rows, cols), np.nan, dtype=np.float32)

    # Calculate center coords for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)

    if len(points) == 0:
        print("step4, vegetation_raster_idw: There are no vegetation points in the current area.")
        return vege_raster, grid_center_xy

    # Convert LAS points to NumPy array for processing
    points_list = np.array(points.xyz)

    # Interpolation
    functions.interpolation_idw_kdtree(points_list, vege_raster, grid_center_xy, search_radius, power)

    return vege_raster, grid_center_xy

def process_laz_files(input_folder, output_folder, ndvi_threshold=0.1, resolution=0.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    laz_files = [f for f in os.listdir(input_folder) if f.endswith(".LAZ")]

    total_start_time = time.time()

    # Iterate over files
    for file_name in tqdm(laz_files, desc="Processing files", unit="file"):
        file_path = os.path.join(input_folder, file_name)
        tile_name = os.path.splitext(file_name)[0]
        output_filename = os.path.join(output_folder, f"CHM_{tile_name}.TIF")

        start_time = time.time()

        # Load LAS data
        with laspy.open(file_path) as las:
            LasData = las.read()

        # Extract vegetation points
        veg_points = extract_vegetation_points(LasData, ndvi_threshold=ndvi_threshold)

        # Create the CHM and save it
        chm_creation(LasData, veg_points, output_filename, resolution=resolution)

        # Calculate time taken for current file
        elapsed_time = time.time() - start_time
        print(f"\nProcessed {file_name} in {elapsed_time:.2f} seconds and saved output to {output_filename}")

        os.remove(file_path)
        print(f"Deleted file: {file_path}")

    # Calculate and print total processing time
    total_elapsed_time = time.time() - total_start_time
    print(f"\nAll files processed in {total_elapsed_time:.2f} seconds.")

In [14]:
input_folder = "data/25GZ2"
output_folder = "output/25GZ2"
process_laz_files(input_folder, output_folder)

Processing files: 100%|██████████| 1/1 [06:51<00:00, 411.64s/file]

File written to 'output/25GZ2\CHM_25GZ2_05.TIF'

Processed 25GZ2_05.LAZ in 411.61 seconds and saved output to output/25GZ2\CHM_25GZ2_05.TIF
Deleted file: data/25GZ2\25GZ2_05.LAZ

All files processed in 411.68 seconds.


In [15]:
input_folder = "data/25BZ1"
output_folder = "output/25BZ1"
process_laz_files(input_folder, output_folder)

Processing files:   4%|▍         | 1/23 [04:19<1:35:04, 259.30s/file]

File written to 'output/25BZ1\CHM_25BZ1_01.TIF'

Processed 25BZ1_01.LAZ in 259.24 seconds and saved output to output/25BZ1\CHM_25BZ1_01.TIF
Deleted file: data/25BZ1\25BZ1_01.LAZ


Processing files:   9%|▊         | 2/23 [05:56<57:23, 163.99s/file]  

File written to 'output/25BZ1\CHM_25BZ1_02.TIF'

Processed 25BZ1_02.LAZ in 97.25 seconds and saved output to output/25BZ1\CHM_25BZ1_02.TIF
Deleted file: data/25BZ1\25BZ1_02.LAZ


Processing files:  13%|█▎        | 3/23 [07:31<44:04, 132.23s/file]

...

File written to 'output/25BZ1\CHM_25BZ1_23.TIF'

Processed 25BZ1_23.LAZ in 202.08 seconds and saved output to output/25BZ1\CHM_25BZ1_23.TIF
Deleted file: data/25BZ1\25BZ1_23.LAZ


Processing files:  96%|█████████▌| 22/23 [1:05:53<03:55, 235.29s/file]

File written to 'output/25BZ1\CHM_25BZ1_24.TIF'

Processed 25BZ1_24.LAZ in 239.78 seconds and saved output to output/25BZ1\CHM_25BZ1_24.TIF
Deleted file: data/25BZ1\25BZ1_24.LAZ


Processing files: 100%|██████████| 23/23 [1:09:20<00:00, 180.89s/file]

File written to 'output/25BZ1\CHM_25BZ1_25.TIF'

Processed 25BZ1_25.LAZ in 207.10 seconds and saved output to output/25BZ1\CHM_25BZ1_25.TIF
Deleted file: data/25BZ1\25BZ1_25.LAZ

All files processed in 4160.51 seconds.


In [16]:
input_folder = "data/25EZ1"
output_folder = "output/25EZ1"
process_laz_files(input_folder, output_folder)

Processing files:   4%|▍         | 1/23 [02:54<1:03:55, 174.33s/file]

File written to 'output/25EZ1\CHM_25EZ1_01.TIF'

Processed 25EZ1_01.LAZ in 174.26 seconds and saved output to output/25EZ1\CHM_25EZ1_01.TIF
Deleted file: data/25EZ1\25EZ1_01.LAZ


Processing files:   9%|▊         | 2/23 [05:28<56:48, 162.33s/file]  

File written to 'output/25EZ1\CHM_25EZ1_02.TIF'

Processed 25EZ1_02.LAZ in 153.87 seconds and saved output to output/25EZ1\CHM_25EZ1_02.TIF
Deleted file: data/25EZ1\25EZ1_02.LAZ


Processing files:  13%|█▎        | 3/23 [08:14<54:43, 164.19s/file]

...

File written to 'output/25EZ1\CHM_25EZ1_23.TIF'

Processed 25EZ1_23.LAZ in 119.22 seconds and saved output to output/25EZ1\CHM_25EZ1_23.TIF
Deleted file: data/25EZ1\25EZ1_23.LAZ


Processing files:  96%|█████████▌| 22/23 [57:53<02:09, 129.96s/file]

File written to 'output/25EZ1\CHM_25EZ1_24.TIF'

Processed 25EZ1_24.LAZ in 92.19 seconds and saved output to output/25EZ1\CHM_25EZ1_24.TIF
Deleted file: data/25EZ1\25EZ1_24.LAZ


Processing files: 100%|██████████| 23/23 [59:26<00:00, 155.06s/file]

File written to 'output/25EZ1\CHM_25EZ1_25.TIF'

Processed 25EZ1_25.LAZ in 92.82 seconds and saved output to output/25EZ1\CHM_25EZ1_25.TIF
Deleted file: data/25EZ1\25EZ1_25.LAZ

All files processed in 3566.30 seconds.


In [17]:
input_folder = "data/25EZ2"
output_folder = "output/25EZ2"
process_laz_files(input_folder, output_folder)

Processing files:   4%|▍         | 1/24 [01:47<41:21, 107.91s/file]

File written to 'output/25EZ2\CHM_25EZ2_01.TIF'

Processed 25EZ2_01.LAZ in 107.86 seconds and saved output to output/25EZ2\CHM_25EZ2_01.TIF
Deleted file: data/25EZ2\25EZ2_01.LAZ


Processing files:   8%|▊         | 2/24 [03:29<38:06, 103.95s/file]

File written to 'output/25EZ2\CHM_25EZ2_02.TIF'

Processed 25EZ2_02.LAZ in 101.14 seconds and saved output to output/25EZ2\CHM_25EZ2_02.TIF
Deleted file: data/25EZ2\25EZ2_02.LAZ


Processing files:  12%|█▎        | 3/24 [05:20<37:34, 107.37s/file]

...

File written to 'output/25EZ2\CHM_25EZ2_23.TIF'

Processed 25EZ2_23.LAZ in 117.82 seconds and saved output to output/25EZ2\CHM_25EZ2_23.TIF
Deleted file: data/25EZ2\25EZ2_23.LAZ


Processing files:  96%|█████████▌| 23/24 [48:00<02:05, 125.78s/file]

File written to 'output/25EZ2\CHM_25EZ2_24.TIF'

Processed 25EZ2_24.LAZ in 112.92 seconds and saved output to output/25EZ2\CHM_25EZ2_24.TIF
Deleted file: data/25EZ2\25EZ2_24.LAZ


Processing files: 100%|██████████| 24/24 [50:00<00:00, 125.03s/file]

File written to 'output/25EZ2\CHM_25EZ2_25.TIF'

Processed 25EZ2_25.LAZ in 120.55 seconds and saved output to output/25EZ2\CHM_25EZ2_25.TIF
Deleted file: data/25EZ2\25EZ2_25.LAZ

All files processed in 3000.71 seconds.


In [18]:
input_folder = "data/25GN2"
output_folder = "output/25GN2"
process_laz_files(input_folder, output_folder)

Processing files:   4%|▍         | 1/24 [02:24<55:13, 144.06s/file]

File written to 'output/25GN2\CHM_25GN2_01.TIF'

Processed 25GN2_01.LAZ in 144.03 seconds and saved output to output/25GN2\CHM_25GN2_01.TIF
Deleted file: data/25GN2\25GN2_01.LAZ


Processing files:   8%|▊         | 2/24 [03:57<41:55, 114.35s/file]

File written to 'output/25GN2\CHM_25GN2_02.TIF'

Processed 25GN2_02.LAZ in 93.53 seconds and saved output to output/25GN2\CHM_25GN2_02.TIF
Deleted file: data/25GN2\25GN2_02.LAZ


Processing files:  12%|█▎        | 3/24 [05:13<33:55, 96.91s/file] 

...

File written to 'output/25GN2\CHM_25GN2_23.TIF'

Processed 25GN2_23.LAZ in 326.30 seconds and saved output to output/25GN2\CHM_25GN2_23.TIF
Deleted file: data/25GN2\25GN2_23.LAZ


Processing files:  96%|█████████▌| 23/24 [1:15:29<05:52, 352.03s/file]

File written to 'output/25GN2\CHM_25GN2_24.TIF'

Processed 25GN2_24.LAZ in 598.52 seconds and saved output to output/25GN2\CHM_25GN2_24.TIF
Deleted file: data/25GN2\25GN2_24.LAZ


Processing files: 100%|██████████| 24/24 [1:20:18<00:00, 200.78s/file]

File written to 'output/25GN2\CHM_25GN2_25.TIF'

Processed 25GN2_25.LAZ in 288.69 seconds and saved output to output/25GN2\CHM_25GN2_25.TIF
Deleted file: data/25GN2\25GN2_25.LAZ

All files processed in 4818.64 seconds.
